In [1]:
from matplotlib import pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = [20, 10]

In [2]:
# Input files
Master_file = '/Users/cvigilv/Dropbox/2018/Data/CC&D mk. 2/chembl23_GS3_v2.mphase_gt_0.txt.co.graphml'

In [3]:
# Configurations
TopX_percentage = 5

In [4]:
import networkx as nx

# Load graph file into graph object
Master_Graph = nx.read_graphml(Master_file)

print('Summary of graph object:\n')
print(nx.info(Master_Graph))
print('Edge attributes:', ', '.join(list((list(Master_Graph.edges(data=True))[0][2]).keys())))
print('Node attributes:', ', '.join(list((list(Master_Graph.nodes(data=True))[0][1]).keys())))

Summary of graph object:

Name: Complete graph
Type: Graph
Number of nodes: 2129
Number of edges: 1169793
Average degree: 1098.9131
Edge attributes: Similarity, Distance, Z_Similarity, Z_Distance, Type
Node attributes: Type, Fold


In [5]:
import seaborn as sns
import pandas as pd
import numpy as np

In [6]:
# Filter graph edges based in previously declared cutoffs

TopX = int(len([n for n, data in Master_Graph.nodes(data = True) if data['Type'] == 'Ligand']) * TopX_percentage/100)
print(TopX)
Ignored_edges = []
Ignored_nodes = []
for n, d in Master_Graph.nodes(data = True):
    if d['Type'] != 'Ligand':
        Ignored_nodes += [n]
    else:
        Ignored_edges += [N for N in 
                          sorted(Master_Graph.edges(n, data = True), 
                                 key = lambda E: E[2]['Similarity'], 
                                 reverse = True) 
                          if N[2]['Type'] == 'LL'][TopX:]
    
Master_Graph.remove_edges_from(Ignored_edges)
Master_Graph.remove_nodes_from(Ignored_nodes)

print('Summary of filtered graph object:\n')
print(nx.info(Master_Graph))
print('Edge attributes:', ', '.join(list((list(Master_Graph.edges(data=True))[0][2]).keys())))
print('Node attributes:', ', '.join(list((list(Master_Graph.nodes(data=True))[0][1]).keys())))

61
Summary of filtered graph object:

Name: Complete graph
Type: Graph
Number of nodes: 1232
Number of edges: 21959
Average degree:  35.6477
Edge attributes: Similarity, Distance, Z_Similarity, Z_Distance, Type
Node attributes: Type, Fold


In [7]:
nx.write_graphml(Master_Graph, Master_file.replace('.graphml', '.Top{}percent.graphml'.format(TopX_percentage)))